# FOODC Densenet 161 LB 0.64

Author - Hard_drive_corrupted (Pulkit Gera & Anchit Gupta)

## To open this notebook on Google Computing platform Colab, click below!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/aicrowd-bot/8a3e4b475f70e48ee7d5dfa168997073)


##Solution
We apply basic augmentation first to the dataset. After that we do the following
+ We train the model on image sizes 128x128 for few epochs 
+ We train again for a few more epochs on image sizes 256x256
+ We apply TTA to improve the solution

We use Densenet 161 with SGD optimizer and Step LR scheduler for training

## Download the files
These include the train test images as well the csv indexing them

In [0]:
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/train_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/test_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/train.csv
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/foodc/v0.1/test.csv


We create directories and unzip the images

In [0]:
!mkdir data
!mkdir data/test
!mkdir data/train
!unzip train_images -d data/train
!unzip test_images -d data/test

## Import necessary packages

In [0]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
from torchvision import models
import torch.optim as optim
import pandas as pd
import numpy as np
import cv2
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline
import time
from tqdm import tqdm

## Loading Data
In pytorch we can directly load our files into torchvision(the library which creates the object) or create a custom class to load data. The class must have `__init__` , `__len__` and `__getitem__` functions. We create a custom dataloader to suit our needs. More info on custom loaders can be read [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [0]:
class FoodData(Dataset):
    def __init__(self,data_list,data_dir = './',transform=None,train=True):
        super().__init__()
        self.data_list = data_list
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
    
    def __len__(self):
        return self.data_list.shape[0]
    
    def __getitem__(self,item):
        if self.train:
          img_name,label = self.data_list.iloc[item]
        else:
          img_name = self.data_list.iloc[item]['ImageId']
        img_path = os.path.join(self.data_dir,img_name)
        img = cv2.imread(img_path,1)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(256,256))
        if self.transform is not None:
            img = self.transform(img)
        if self.train:
          return {
              'gt' : img,
              'label' : torch.tensor(label)

          }
        else:
          return {
              'gt':img
          }
        

We first convert the data labels into encodings using Label Encoders. This basically converts labels into number encodings. This is an important step as without it we cannot train our network

In [0]:
train = pd.read_csv('train.csv')
le = preprocessing.LabelEncoder()
targets = le.fit_transform(train['ClassName'])
ntrain = train
ntrain['ClassName'] = targets

In [0]:
ntrain

,ImageId,ClassName
0,f27632d7e5.jpg,55
1,efa87919ed.jpg,41
2,4f169e8c8d.jpg,12
3,a6956654bf.jpg,44
4,d99ce8c3bf.jpg,23
...,...,...
9318,ba8233c7d2.jpg,7
9319,2090043907.jpg,58
9320,8762d1cefd.jpg,14
9321,28e7439245.jpg,12


We load our train data and some necessary augementations like converting to PIL image, converting to tensors and normalizing them across channels. We can add more augementations such as `Random Flip`, `Random Rotation`, etc more on which can be found [here](https://pytorch.org/docs/stable/torchvision/transforms.html)

In [0]:
transforms_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize( mean = np.array([0.485, 0.456, 0.406]),
    std = np.array([0.229, 0.224, 0.225]))
])
train_path = 'data/train/train_images'
train_data = FoodData(data_list= ntrain,data_dir = train_path,transform = transforms_train)

## EDA
Let us do some exploratory data analysis. The idea is to see the class distribution, how the images are and much more. 

## Split Data into Train and Validation
Now we want to see how well our model is performing, but we dont have the test data labels with us to check. What do we do ? So we split our dataset into train and validation. The idea is that we test our classifier on validation set in order to get an idea of how well our classifier works. This way we can also ensure that we dont [overfit](https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/) on the train dataset. There are many ways to do validation like [k-fold](https://machinelearningmastery.com/k-fold-cross-validation/),[leave one out](https://en.wikipedia.org/wiki/Cross-validation_(statistics), etc  

We also make `dataloaders` which basically create minibatches of dataset which are used in each epoch

In [0]:
batch = 4
valid_size = 0.2
num = train_data.__len__()
# Dividing the indices for train and cross validation
indices = list(range(num))
np.random.shuffle(indices)
split = int(np.floor(valid_size*num))
train_idx,valid_idx = indices[split:], indices[:split]

#Create Samplers
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(train_data, batch_size = batch, sampler = train_sampler)
valid_loader = DataLoader(train_data, batch_size = batch, sampler = valid_sampler)


Here we load test images. Note: This file will not have any labels with it

In [0]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize( mean = np.array([0.485, 0.456, 0.406]),
    std = np.array([0.229, 0.224, 0.225]))
])
test_path = 'data/test/test_images'
test = pd.read_csv('test.csv')
test_data = FoodData(data_list= test,data_dir = test_path,transform = transforms_test,train=False)

test_loader = DataLoader(test_data, batch_size=batch, shuffle=False)


Here we check if we have a GPU or not. If we have we just need to shift our data and model to GPU for faster computations.

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
    

cuda:0


## Define the Model
Now we come to the juicy part. We define our model here. We need to create a class with `__init__` and `forward` functions which define the layers and forward pass respectively. We can also load pretrained models and freeze their layers and add more layers on top of it, to train them. More on pretrained models with pytorch [here](https://pytorch.org/docs/stable/torchvision/models.html) and making models [here](https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/).

In [0]:
train_sampler.__len__()

7459

In [0]:
dataloaders = {}
dataset_sizes = {}
dataloaders['train'] = train_loader
dataloaders['val'] = valid_loader
dataset_sizes['train'] = train_sampler.__len__()
dataset_sizes['val'] = valid_sampler.__len__()

In [0]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):

    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index,:]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(y_a, y_b, lam):
    return lambda criterion, pred: lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


In [0]:
from torch.autograd import Variable
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in tqdm(dataloaders[phase], position=0, leave=True):
                inputs = data['gt'].squeeze(0).to(device)
                labels = data['label'].to(device)
                  
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)                    
                    _, preds = torch.max(outputs, 1)

                    # loss = criterion(outputs, labels)
                    loss = criterion(outputs,labels) 

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                # running_corrects += torch.sum(preds == labels.data)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'best_model_so_far.pth')

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
import pretrainedmodels
model_name = 'xception' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /root/.cache/torch/checkpoints/xception-43020ad28.pth


In [0]:
import copy
model_ft = models.densenet161(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 61)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [0]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

  0%|          | 0/1865 [00:00<?, ?it/s]

Epoch 0/4
----------


  0%|          | 0/466 [00:00<?, ?it/s]

train Loss: 2.7353 Acc: 0.3289


100%|██████████| 466/466 [01:55<00:00,  4.04it/s]


...


Epoch 4/4
----------


  0%|          | 0/466 [00:00<?, ?it/s]

train Loss: 1.4725 Acc: 0.5789


100%|██████████| 466/466 [01:54<00:00,  4.06it/s]


val Loss: 1.5856 Acc: 0.5810

Training complete in 119m 15s
Best val Acc: 0.581009


In [0]:
torch.cuda.empty_cache()

## Predict on Validation
Now we predict our trained model on the validation set and evaluate our model

In [0]:
!pip install ttach

In [0]:
import ttach as tta
transforms = tta.Compose(
    [
	# tta.FiveCrops(128,128),        	
	tta.HorizontalFlip(),
	tta.VerticalFlip(),
	# tta.FiveCrops(128,128),
  tta.Rotate90(angles=[0, 90]),
        #tta.Scale(scales=[1, 2, 4]),
	        
    ]
)
model_ft.load_state_dict(torch.load('best_model_so_far.pth'))
model_ft.eval()
tta_model = tta.ClassificationTTAWrapper(model_ft, transforms,merge_mode='mean')

In [0]:


correct = 0
total = 0
pred_list = []
correct_list = []
with torch.no_grad():
    for images in tqdm(valid_loader):
        data = images['gt'].squeeze(0).to(device)
        target = images['label'].to(device)
        outputs = tta_model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          pred_list.append(i)
        tg = target.detach().cpu().numpy()
        for i in tg:
          correct_list.append(i)
        correct += (predicted == target).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

## Evaluate the Performance
We use the same metrics as that will be used for the test set.  
[F1 score](https://en.wikipedia.org/wiki/F1_score) and [Log Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html) are the metrics for this challenge

In [0]:
from sklearn.metrics import f1_score,precision_score,log_loss   
print("F1 score :",f1_score(correct_list,pred_list,average='micro')*100)

F1 score : 65.71888412017167


## Predict on test set
Time for the moment of truth! Predict on test set and time to make the submission.

In [0]:
model_ft.load_state_dict(torch.load('best_model_so_far.pth'))
model_ft.eval()

preds = []
with torch.no_grad():
    for images in tqdm(test_loader):
        data = images['gt'].squeeze(0).to(device)
        outputs = model_ft(data)
        _, predicted = torch.max(outputs.data, 1)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          preds.append(i)


## Save it in correct format

In [0]:
# Create Submission file        
df = pd.DataFrame(le.inverse_transform(preds),columns=['ClassName'])
df.to_csv('submission.csv',index=False)

## To download the generated in collab csv run the below command

In [0]:
from google.colab import files
files.download('submission.csv') 

### Go to [platform](https://www.aicrowd.com/challenges/aicrowd-blitz-may-2020/problems/foodc). Participate in the challenge and submit the submission.csv.